# Setup notebook
Don't forget to activate the GPU usage in the notebook parameter. Click on `Modify -> Notebook parameter`. Under the `Hardware acceleration` section, choose `GPU`

# Clone or pull Valuenet

In [1]:
!git clone https://github.com/brunnurs/valuenet.git
# !git pull

Cloning into 'valuenet'...
remote: Enumerating objects: 3043, done.
remote: Counting objects: 100% (1646/1646), done.
remote: Compressing objects: 100% (748/748), done.
remote: Total 3043 (delta 956), reused 1484 (delta 842), pack-reused 1397
Receiving objects: 100% (3043/3043), 21.60 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (1889/1889), done.


# Setup and install dependencies


In [1]:
%cd valuenet/
!pip install ipywidgets
!spacy download en_core_web_sm
!pip install -r requirements.txt

/content/valuenet
     |████████████████████████████████| 13.6 MB 78 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Verify GPU

In [2]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi

Thu Sep 23 13:43:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Train Model
Here also, we follow the [user manual from Valuenet](https://github.com/brunnurs/valuenet).

## Training data

The data that will be used for training is located in the files

    valuenet/data/dataset/train.json
    valuenet/data/dataset/dev.json

with dataset being either hack_zurich or spider.



### Option A: Train with Spider dataset

In [3]:
data_set = 'spider'

### Option B: Train with Custom Data only

Create your custom data and preprocess it following the steps of [preprocess_custom_data-01.ipynb](https://github.com/hack-with-admin-ch/aws-sagemaker-notebook-valuenet/blob/main/preprocess_custom_data-01.ipynb). 

Then, set dataset to 'hack_zurich' (as the cell below). 

In [4]:
data_set = 'hack_zurich'

### Option C: Train with Spider Dataset and the Custom Data

In this case, merge your custom data and spider data together in one json file for train, one for dev and one for the tables:
Concatenate
- valuenet/data/hack_zurich/train.json with valuenet/data/spider/train.json
- valuenet/data/hack_zurich/dev.json with valuenet/data/spider/dev.json
- valuenet/data/hack_zurich/original/tables.json with valuenet/data/spider/original/tables.json

and save the merged files in hack_zurich valuenet/data/hack_zurich/train.json, valuenet/data/hack_zurich/dev.json and valuenet/data/hack_zurich/original/tables.json.

As fine-tuning was never experimented with the Valuenet codebase, it is the safest way to train a model which will perform better on custom data.

In [5]:
data_set = 'hack_zurich'

## Training procedure

**Hyperparameters**:
Here, we set a few parameters but do not hesitate to look at [training parameters](https://github.com/brunnurs/valuenet/blob/hackzurich/src/config.py) if you want to specify other settings.

Train from scratch (with exception of the transformer encoder, which is already pre-trained). Fine-tuning was never implemented with valuent, feel free to try !

In [6]:
exp_name = "train-01" # Name of the training (saved in wandb and folder name in experiments)
batch_size = 8 # make it smaller if you encounter memory limitations 
num_epochs = 5

Be sure you have uploaded the dev.json and train.json files to `valuenet/data/hack_zurich` before performing the following steps

In [9]:
%run src/main.py --exp_name {exp_name} --data_set {data_set} --cuda --batch_size {batch_size} --num_epochs {num_epochs}

wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


*** parsed configuration from command line and combine with constants ***
argument: cuda=True
argument: encoder_pretrained_model=facebook/bart-base
argument: max_seq_length=1024
argument: column_pointer=True
argument: embed_size=300
argument: hidden_size=300
argument: action_embed_size=128
argument: att_vec_size=300
argument: type_embed_size=128
argument: col_embed_size=300
argument: readout=identity
argument: column_att=affine
argument: dropout=0.3
argument: exp_name=train-01
argument: seed=90
argument: toy=False
argument: data_set=hack_zurich
argument: batch_size=8
argument: num_epochs=5.0
argument: lr_base=0.001
argument: lr_connection=0.0001
argument: lr_transformer=2e-05
argument: scheduler_gamma=0.5
argument: max_grad_norm=1.0
argument: clip_grad=5.0
argument: loss_epoch_threshold=50
argument: sketch_loss_weight=1.0
argument: run_spider_evaluation_after_epoch=False
argument: beam_size=5
argument: decode_max_time_step=40
argument: data_dir=data/hack_zurich
argument: model_output_d

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Not all values found during pre-processing for question "How high is the total share of electric vehicles within the newly registered cars in the Canton Zurich in the year 2002?". Replace values with dummy to make query fail



Evaluating: 100%|██████████| 20/20 [00:38<00:00,  1.92s/it]


Not all values found during pre-processing for question "Which region had the lowest share of electric vehicles within the newly registered cars in the year 2002?". Replace values with dummy to make query fail
in 0 times we found the correct results in another beam (failing queries: 152)


100%|██████████| 5/5 [12:26<00:00, 149.33s/it]

Epoch: 5    Sketch-Accuracy: 0.5714285714285714     Accuracy: 0.012987012987012988


**Results and final model**:
After training, you will be able to find the ground truth data, output of the model, evaluation results and intermediary data in the folder `valuenet/experiments/{exp_name}_date_time`.
If the accuracy was better than the last best accuracy, the model is also saved in the folder as `"best_model.pt"`.